In [1]:
##!pip install torch
##!pip install transformers
##from transformers import AutoTokenizer
##!pip install fastparquet
##tokenizer = AutoTokenizer.from_pretrained("cahya/bert-base-indonesian-NER",  truncation=True)
import warnings
warnings.filterwarnings("ignore")

import os
import re

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from collections import defaultdict
from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from torch.nn import functional as F
import torch
##torch.cuda.is_available()
##device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [10]:
DEBUG = False
TARGET = 'POI/street'
POI = 'POI'
STREET = 'street'
SEED = 2021

In [11]:
df = pd.read_csv('C:/Users/Ale/Desktop/jupyter_folder/FYP/train.csv')
test = pd.read_csv('C:/Users/Ale/Desktop/jupyter_folder/FYP/test.csv')

In [12]:
train, val = train_test_split(df, shuffle=True, random_state=SEED)

In [13]:
train['split'] = "train"
val['split'] = "val"
test['split'] = 'test'

In [14]:
def naive_tokenize(x):
    if type(x) == str:
        return re.split('(\W)', x)
    return x

In [15]:
test['POI/street'] = np.nan
train.shape, val.shape, test.shape
data = train.append(val).append(test).reset_index(drop=True) 

In [16]:
print("DEBUG", DEBUG)
if DEBUG:
    data = data.sample(50000)

DEBUG False


In [17]:
data[POI] = data[TARGET].str.split('/', n = 2, expand = True)[0]
data[STREET] = data[TARGET].str.split('/', n = 2, expand = True)[1]

In [18]:
data['address_list'] = data['raw_address'].apply(naive_tokenize)
data['POI_list'] = data[POI].apply(naive_tokenize)
data['street_list'] = data[STREET].apply(naive_tokenize)

In [19]:
from collections import defaultdict

def generate_n_gram(_list, n=2):
    if len(_list) < n:
        return []
    else:
        return [[_list[i+p] for p in range(n)] for i in range(len(_list)-(n-1))]

In [20]:
def bigram_maping(tokens, poi_tokens, target_bigram):
    pair_token = generate_n_gram(tokens)
    pair_poi = generate_n_gram(poi_tokens)
    for poi1, poi2 in pair_poi:
        for token1, token2 in pair_token:
            if poi1.startswith(token1) and poi2.startswith(token2):
                target_bigram[(token1, token2)][(poi1,poi2)] += 1

In [21]:
def trigram_maping(tokens, poi_tokens, target_trigram):
    thrice_token = generate_n_gram(tokens, 3)
    thrice_poi = generate_n_gram(poi_tokens, 3)
    for poi1, poi2, poi3 in thrice_poi:
        for token1, token2, token3 in thrice_token:
            if poi1.startswith(token1) and poi2.startswith(token2) and poi3.startswith(token3):
                target_trigram[(token1, token2, token3)][(poi1,poi2, poi3)] += 1

In [22]:
train = data[data['split'] == 'train']

In [23]:
bigram_poi = defaultdict(lambda: defaultdict(lambda :0))
_ = train[['address_list', 'POI_list']].progress_apply(lambda x: bigram_maping(*x, bigram_poi), axis=1)

100%|██████████| 225000/225000 [00:04<00:00, 49866.92it/s]


In [24]:
trigram_poi = defaultdict(lambda: defaultdict(lambda :0))
_ = train[['address_list', 'POI_list']].progress_apply(lambda x: trigram_maping(*x, trigram_poi), axis=1)

100%|██████████| 225000/225000 [00:04<00:00, 48333.02it/s]


In [25]:
len(bigram_poi), len(trigram_poi)

(55519, 127283)

In [26]:
bigram_street = defaultdict(lambda: defaultdict(lambda :0))
_ = train[['address_list', 'street_list']].progress_apply(lambda x: bigram_maping(*x, bigram_street), axis=1)

100%|██████████| 225000/225000 [00:04<00:00, 45499.29it/s]


In [27]:
trigram_street = defaultdict(lambda: defaultdict(lambda :0))
_ = train[['address_list', 'street_list']].progress_apply(lambda x: trigram_maping(*x, trigram_street), axis=1)

100%|██████████| 225000/225000 [00:04<00:00, 48976.34it/s]


In [28]:
def join_dict(dict1, dict2):
    dict1 = dict(dict1)
    dict2 = dict(dict2)
    
    _join_dict = {k:dict(dict1[k]) for k in dict1}
    for k in dict2:
        if k not in _join_dict:
            _join_dict[k] = dict(dict2[k])
        else:
            for kk in dict2[k]:
                if kk not in _join_dict[k]:
                    _join_dict[k][kk] = dict2[k][kk]
                else:
                    _join_dict[k][kk] += dict2[k][kk]
    return _join_dict

In [29]:
import pickle
with open('bigram_street.pkl', 'wb') as f:
    pickle.dump(join_dict(bigram_street, {}), f)
    
with open('trigram_street.pkl', 'wb') as f:
    pickle.dump(join_dict(trigram_street, {}), f)

In [30]:
import pickle
with open('bigram_POI.pkl', 'wb') as f:
    pickle.dump(join_dict(bigram_poi, {}), f)
    
with open('trigram_POI.pkl', 'wb') as f:
    pickle.dump(join_dict(trigram_poi, {}), f)

In [31]:

def get_maping(maping, token):
    if token not in maping:
        return token
    try:
        max_token = max(maping[token], key=maping[token].get)
        if maping[token][max_token] > 3:
            return max_token
        else:
            return token
    except:
        return token
    

def reformat_target(addr, bigram_target, trigram_target):
    if len(addr) > 2:
        for idx in range(len(addr) -2):
            a, b, c = addr[idx], addr[idx+1], addr[idx+2]
            a, b, c = get_maping(trigram_target, (a, b, c))
            addr[idx] = a
            addr[idx+1] = b
            addr[idx+2] = c

#     if len(addr) > 1:
#         for idx in range(len(addr) -1):
#             a, b = get_maping(bigram_target, (addr[idx], addr[idx+1]))
#             addr[idx] = a
#             addr[idx+1] = b
    return addr

In [35]:
import nltk
from nltk.metrics import distance

OTHER = 'O'

class LabelGenerator:
    
    def __init__(self):
        self.maping_poi = defaultdict(
            lambda: defaultdict(
                lambda: 0
            )
        )
        self.inv_maping_poi = defaultdict(
            lambda: defaultdict(
                lambda: 0
            )
        )

        self.maping_street = defaultdict(
            lambda: defaultdict(
                lambda: 0
            )
        )
        self.inv_maping_street = defaultdict(
            lambda: defaultdict(
                lambda: 0
            )
        )
    
    def calculate_score(self, token1, token2):
        scores = []
        for idx, v1 in enumerate(token1):
            v2 = token2[idx]
            ed = distance.edit_distance(v1, v2)
            scores.append(ed)
        return np.mean(scores)
            
    def generate_score(self, tokens, target, debug=False):
        scores = {
            'idx':[],
            'scores': []
        }
        if debug:
            print(tokens)
            print(target)
        L_target = len(target)
        L_tokens = len(tokens)
        
        for i in range(0, L_tokens - L_target + 1):
            if debug:
                print(i)
            scores['idx'].append(set(range(i,i+L_target)))
            scores['scores'].append(self.calculate_score(tokens[i:i+L_target], target))
        return pd.DataFrame(scores)

    def generate_single_label(self, address_token, street_token, POI_token, debug=False):
        
        labels = [OTHER for _ in address_token]
    
        curstreet = 0
        curpoi = 0
        L_addrs = len(address_token)
        L_street = len(street_token)
        L_poi = len(POI_token)
        
        if L_street == 0 and L_poi == 0:
            return labels

#         if L_street == 0 or L_poi == 0:
#             return old_generate_label(address_token, street_token, POI_token, debug=False)
            
        street_score = self.generate_score(address_token, street_token, debug)
        POI_score = self.generate_score(address_token, POI_token, debug)
                
        street_score = street_score.sort_values('scores').reset_index(drop=True)
        POI_score = POI_score.sort_values('scores').reset_index(drop=True)
        
        poi_idx = 0
        street_idx = 0
        if debug:
            print(L_addrs, L_street, L_poi)
            print('street', street_score.shape)
            print(street_score)
            print('poi', POI_score.shape)
            print(POI_score)            
        
        
        street_score_shape = street_score.shape[0]
        poi_score_shape = POI_score.shape[0]
        
        
        if L_street == 0:
            street_idx = -1
            street_score_shape = 0
        
        if L_poi == 0:
            poi_idx = -1
            poi_score_shape = 0
        
        
        while poi_idx < poi_score_shape and street_idx < street_score_shape:
            
            if poi_score_shape == 0:
                poi_labels = set()
            else:
                poi_labels = POI_score['idx'].iloc[poi_idx]
                
            if street_score_shape == 0:
                street_labels = set()
            else:
                street_labels = street_score['idx'].iloc[street_idx]

            if debug:
                print(street_score_shape, street_labels, poi_score_shape,poi_labels,  len(poi_labels.intersection(street_labels)))
            
            if len( poi_labels.intersection(street_labels)) == 0:
                if debug:
                    print('masuk')
                list_poi_labels = list(sorted(poi_labels))
                list_street_labels = list(sorted(street_labels))

                current = None
                
                cur_poi_idx = 0
                cur_street_idx = 0

                for idx, _ in enumerate(labels):
                    if idx in poi_labels:
                        

                        self.maping_poi[address_token[idx]][POI_token[cur_poi_idx]] += 1
                        self.inv_maping_poi[POI_token[cur_poi_idx]][address_token[idx]] += 1                           
                        cur_poi_idx += 1
                        
                        if current != 'POI':
                            current = 'POI'
                            labels[idx] = 'B-POI'
                        else:
                            labels[idx] = 'I-POI'
                    elif idx in street_labels:
                        self.maping_street[address_token[idx]][street_token[cur_street_idx]] += 1
                        self.inv_maping_street[street_token[cur_street_idx]][address_token[idx]] += 1                           
                        cur_street_idx += 1
                        
                        
                        if current != 'STREET':
                            current = 'STREET'
                            labels[idx] = 'B-STREET'
                        else:
                            labels[idx] = 'I-STREET'
                    else:
                        current = None
                return labels
            if len(poi_labels) < len(street_labels):
                poi_idx += 1
            else:
                street_idx += 1
        
        return []

In [36]:
lg_train = LabelGenerator()
lg_val = LabelGenerator()

In [37]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
  
tokenizer = AutoTokenizer.from_pretrained("cahya/bert-base-indonesian-NER",  truncation=False)

model = AutoModelForTokenClassification.from_pretrained("cahya/bert-base-indonesian-NER")

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [38]:
from tqdm import tqdm
tqdm.pandas()

In [39]:
import pickle

In [40]:
data['address_list'][2]

['kencana',
 ' ',
 'ayam',
 ' ',
 'mrothol',
 ',',
 '',
 ' ',
 'raya',
 ' ',
 'kenc',
 ',',
 '',
 ' ',
 '2',
 ' ',
 'tanah',
 ' ',
 'sereal']

In [41]:
model = model.to(device)

In [42]:
# logits = output.logits
# softmax = F.softmax(logits, dim = -1)

In [43]:
def reconstruct_tokens(sent, prediction):
    sent = sent[1:-1]
    prediction = prediction[1:-1]
    
    sentences = []
    predictions = []

    last_token = ""
    las_predict = []

    for idx, token in enumerate(sent):
        if token.startswith("##"):
            predictions[-1].append(prediction[idx])
            sentences[-1] += token.lstrip("##")
        else:
            sentences.append(token)
            predictions.append([prediction[idx]])
    return sentences, [np.mean(p, axis=0) for p in predictions]

In [44]:
label_list = model.config.id2label

In [45]:
worklist = data['raw_address'].tolist()
batchsize = 32

enumerator = list(range(0, len(worklist), batchsize))
tot = 0

assert sum([len(worklist[i:i+batchsize]) for i in enumerator]) == data.shape[0]

In [46]:
from tqdm import tqdm

In [47]:
model = model.to(device)

In [48]:
device

'cpu'

In [49]:
tokens = []
predictions = []
for i in tqdm(enumerator, total=len(enumerator)):
    works = worklist[i:i+batchsize]
    batch = tokenizer(works, padding=True, return_tensors="pt").to(device)
    
    output = model(**batch)
    
    for sent_idx, _  in enumerate(batch['input_ids']):
        logits = output.logits
        softmax = F.softmax(logits, dim = -1)


        if device == 'cuda:0':
                prediction = softmax.cpu()[sent_idx][torch.where(batch['attention_mask'][sent_idx] > 0)].detach().numpy().astype(np.float16)
        else:
            prediction = softmax[sent_idx][torch.where(batch['attention_mask'][sent_idx] > 0)].detach().numpy().astype(np.float16)
        token = [x for idx, x in enumerate(tokenizer.convert_ids_to_tokens(batch['input_ids'][sent_idx])) if batch['attention_mask'][sent_idx][idx]]
        

        token, prediction = reconstruct_tokens(token, prediction)
        assert len(prediction) == len(token)
        tokens.append(token)
        predictions.append(prediction)

100%|██████████| 10938/10938 [4:24:32<00:00,  1.45s/it] 


In [50]:
tokens[0]

['ace', 'asw', ',', 'kenanga', ',', 'baloi', 'indah']

In [51]:
np.sum(predictions[0], axis=1)

array([1., 1., 1., 1., 1., 1., 1.], dtype=float16)

In [52]:
tokens[0]

['ace', 'asw', ',', 'kenanga', ',', 'baloi', 'indah']

In [53]:
import pickle
with open('bert-base-predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f)


In [54]:
data['tokens'] = tokens

In [57]:
data.to_parquet('df.pqt')